In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
import pickle


In [2]:
df=pd.read_csv("Churn_Modelling.csv")

In [3]:
df.drop(columns=['CustomerId','RowNumber','Surname'],axis=1,inplace=True)

In [4]:

label_encoder=LabelEncoder()
df['Gender']=label_encoder.fit_transform(df['Gender'])



In [5]:
ohe_encoder=OneHotEncoder()
ohe_encoder=OneHotEncoder(handle_unknown='ignore')
geo_encoded=ohe_encoder.fit_transform(df[['Geography']]).toarray()

In [6]:
geo_encoded_df=pd.DataFrame(geo_encoded,columns=ohe_encoder.get_feature_names_out(['Geography']))

In [7]:
df=pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [8]:
X=df.drop('Exited',axis=1)
y=df['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
with open('label_encoder_gender.pkl','wb') as file :
    pickle.dump(label_encoder,file)
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(ohe_encoder,file)
with open('scaler.pkl','wb')as file:
    pickle.dump(scaler,file)



In [9]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model


In [10]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)





In [14]:
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [15]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
from tensorflow.keras.callbacks  import EarlyStopping ,TensorBoard
log_dir="logs/fit"+datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [ ]:
##set up Early Stopping 
early_stoppping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
history=create_model(X_train,y_train,validation_data=(X_test,y_test),epochs=,
                  callbacks=[tensorflow_callback,early_stoppping_callback])